# Plotting
`scipp` offers a number of different ways to plot data contained in a `Dataset`.

In [ ]:
import numpy as np
import scipp as sc
from scipp import Dim

There are currently two different backends for plotting. The default uses the `plotly` package to render interactive graphs, well suited to `jupyter` notebooks, while the second employs the more classical and widespread `matplotlib` library.

Here we switch to the `matplotlib` backend, as it `plotly` currently does not work for the documentation pages on `ReadTheDocs`.

In [ ]:
sc.plot_config.backend = "matplotlib"

## 1D line plots
A simple line plot can be produced as follows

In [ ]:
d = sc.Dataset()
N = 50
d.coords[Dim.Tof] = sc.Variable([Dim.Tof], values=np.arange(N).astype(np.float64), unit=sc.units.us)
d['Sample'] = sc.Variable([Dim.Tof], values=10.0*np.random.rand(N), unit=sc.units.counts)
sc.plot(d)

Error bars are also shown in variances are present in the data.

In [ ]:
d['Sample'].variances = np.random.rand(N)
sc.plot(d)

If a `Dataset` contains more than one 1D variable with the same coordinates, they are plotted on the same axes

In [ ]:
d['Background'] = sc.Variable([Dim.Tof], values=5.0*np.random.rand(N), unit=sc.units.m)
d

In [ ]:
sc.plot(d)

But one can always plot just a subset of the `Dataset`:

In [ ]:
sc.plot(d['Background'])

## 1D histograms
Histograms are automatically generated if the coordinate is bin edges:

In [ ]:
d['Histogram'] = sc.Variable([Dim.Tof], values=20.0*np.random.rand(N-1), unit=sc.units.counts)
sc.plot(d['Histogram'])

and with error bars

In [ ]:
d['Histogram'].variances = 5.0*np.random.rand(N-1)
sc.plot(d['Histogram'])

## Plotting 2D data
2D variables can be shown as images with a colormap

In [ ]:
N = 100
M = 50
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = N/20.0
c = M/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2)
a = np.sin(r)
d1 = sc.Dataset()
d1.coords[Dim.X] = sc.Variable([Dim.X], values=xx, unit=sc.units.m)
d1.coords[Dim.Y] = sc.Variable([Dim.Y], values=yy, unit=sc.units.m)
d1['Signal'] = sc.Variable([Dim.Y, Dim.X], values=a, unit=sc.units.counts)
sc.plot(d1)

If variances are present, they are not displayed by default, but they can be shown alongside the data values by using `show_variances=True`.

In [ ]:
d1['Signal'].variances = np.random.rand(M, N)+(x==y)
sc.plot(d1, show_variances=True)

If interactive plotting is enabled in the `jupyter` notebook (either using the `plotly` backend or running `%matplotlib notebook` at the start of the notebook), when zooming on either the values or the variances panel will also update the counterpart panel.

## Plotting 3D data

## N-dimensional data